In [1]:
# 手始めに、BERTのクラス分類で動詞を予測することとする

import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

import pickle


/home/takuya/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/takuya/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# データの前処理(データローダに入力できる形に整形)
# 分類する動詞のリスト
verb_list = ['ESTIMATE', 'COMINGOUT', 'DIVINATION', 'GUARD', 'VOTE',
            'ATTACK', 'DIVINED', 'IDENTIFIED', 'GUARDED', 'VOTED',
            'ATTACKED', 'AGREE', 'DISAGREE', 'Skip', 'Over' ] # REVIEW: Skip, Overをuppercaseにする必要があるかどうか

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# 文章をトークンに変換するトークナイザーの読み込み
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# ファインチューニングで学習するためミニバッチ生成用データローダ
# TODO: データを読み込んで前処理をする
# 1. textをトークンに変換
# 2. トークンにラベルを追加
# 3. dataset_for_loaderに追加
max_length = 128
dataset_for_loader = []

# データを取得
data_set_plain = pickle.load(open('dataset.pkl', 'rb'))
for data in data_set_plain:
    encoding = tokenizer(
            data[1],
            max_length=max_length, 
            padding='max_length',
            truncation=True
        )
    encoding['labels'] = verb_list.index(data[0].split()[1])
    encoding = { k: torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)

# #以下実行検証用学習データ
# test_plain_dataset = [
# ['ESTIMATE','私はAgent[10]が騎士だと推測する'],
# ['COMINGOUT','Agent[01]はAgent[03]が狂人だとカミングアウトする'],
# ['Over','Over'],
# ['COMINGOUT','Agent[01]が占いだとカミングアウトする'],
# ['COMINGOUT','Agent[01]はAgent[01]が占い師だとカミングアウトする'],
# ['DIVINED','私は占った結果Agent[01]は白だった'],
# ['DIVINED','Agent[01]が占った結果Agent[02]は人狼だった'],
# ['GUARD','Agent[01]を護衛する'],
# ]
# for data in test_plain_dataset:
#     encoding = tokenizer(
#             data[1],
#             max_length=max_length, 
#             padding='max_length',
#             truncation=True
#         )
#     encoding['labels'] = verb_list.index(data[0])
#     encoding = { k: torch.tensor(v) for k, v in encoding.items() }
#     dataset_for_loader.append(encoding)

print(dataset_for_loader[0])


Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

{'input_ids': tensor([    2, 16204,  1865,  4314,  6420,  4118,     9, 16204,  1865,  4314,
         6420,  4118,    14, 17703,    75,    13, 18148,   123,  7416,    34,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [3]:
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
) 
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [26]:
class BertForSequenceClassification_pl(pl.LightningModule):
        
    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()
        
        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters() 

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )
        
    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss
        
    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [27]:
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='jp_to_prompt_model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    gpus=1, 
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=len(verb_list), lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val) 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1927: PossibleUserWarning: The number of training batches (21)

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [29]:
# 検証データで確認
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  /robot-qnap-1/okubo/dl_remote/work/aiwolf/jp_to_prompt_model/epoch=9-step=210.ckpt
ベストモデルの検証データに対する損失:  tensor(0.0438, device='cuda:0')


In [18]:
# 学習推移の確認
%load_ext tensorboard
%tensorboard --logdir ./

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9220), started 0:01:54 ago. (Use '!kill 9220' to kill it.)

In [30]:
# テストデータで確認
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1444: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /robot-qnap-1/okubo/dl_remote/work/aiwolf/jp_to_prompt_model/epoch=9-step=210.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
Loaded model weights from checkpoint at /robot-qnap-1/okubo/dl_remote/work/aiwolf/jp_to_prompt_model/epoch=9-step=210.ckpt
/home/robot_dev3/anaconda3/envs/aiwolf/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │            1.0            │
└───────────────────────────┴───────────────────────────┘

Accuracy: 1.00


In [31]:
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
) 

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('./model_transformers') 

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [5]:
#保存したモデルが取得できるか確認
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers'
)
bert_sc = bert_sc.cuda()

In [11]:
# 学習したモデルを検証
text_list = [
    "Agent[05]はAgent[05]がボディーガードだとカミングアウトする",
    "Agent[15]が襲われたAgent[10]を霊媒すると白だった",
    "Agent[09]はAgent[09]が占い師だとカミングアウトする",
    "Agent[05]はAgent[05]がボディーガードだと自白する",
    "Agent[05]はAgent[05]がボディーガードだと暴露する",
    "Agent[01]はAgent[01]が人狼だと思う",
    "Agent[01]を吊るすべき",
    "私はAgent[01]に投票しようと思います",
    "Agent[01]がAgent[02]を護衛する",
    "Agent[01]がAgent[02]を守った", # <-汎化性能が低い。学習データに”護衛した”しかないからだと思われる
    "私はAgent[01]の考えに賛成です" # <-学習データにAGREEがないので、うまく行かない
]

# データの符号化
encoding = tokenizer(
        text_list,
        max_length=max_length, 
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

encoding = { k: v.cuda() for k, v in encoding.items() }

# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1).cpu().numpy() # スコアが最も高いラベル
labels_predicted = [verb_list[i] for i in labels_predicted] # ラベルをラベル名に変換
print(labels_predicted)


['COMINGOUT', 'IDENTIFIED', 'COMINGOUT', 'COMINGOUT', 'COMINGOUT', 'ESTIMATE', 'ESTIMATE', 'VOTE', 'GUARDED', 'VOTE', 'ESTIMATE']
